In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

#from tqdm import tqdm

import pickle
from scipy.sparse import *

import get_raintg_X
from sklearn.decomposition import NMF

ImportError: No module named tqdm

In [2]:
data_path = '../data/'
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')

X = train.drop(['source_system_tab', 'source_screen_name', 'source_type'],\
           axis=1)
Y = train['target'].values
X_test = test.drop(['id','source_system_tab',\
                    'source_screen_name', 'source_type'], axis=1)

data = pd.concat([X, X_test], axis=0)
data.msno = pd.Categorical(data.msno)
data.msno = data.msno.cat.codes
data.song_id = pd.Categorical(data.song_id)
data.song_id = data.song_id.cat.codes

num_songs = len(data.song_id.unique())
num_users = len(data.msno.unique())

pos_X = data[data.target == 1]

In [12]:
test = data[pd.isnull(data.target)]

In [18]:
print (test.tail())

          msno  song_id  target
2556785  19206   396079     NaN
2556786  25753   146146     NaN
2556787  25753   409892     NaN
2556788  25753   226660     NaN
2556789  25753   288315     NaN


In [34]:
row = pos_X.msno.append(test.msno)
col = pos_X.song_id.append(test.song_id)
content = np.concatenate(( np.ones(len(pos_X)) , np.empty(len(test)) * np.nan))

In [37]:
X = csc_matrix((content, (row, col)), \
               shape=(num_users, num_songs)).toarray()

In [43]:
print(np.shape(X))

(34403, 419839)


In [42]:
# P (U * k),  Q(I * k)
K = 10

In [44]:
model = NMF(n_components=4, init='random', random_state=0)

In [ ]:
W = model.fit_transform(X)